In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
bookings = pd.read_csv(r"C:\Users\payoj.jain/Documents/projects/Europe Revenue estimation/data/Booking_level_data.csv")

In [3]:
bookings.head()

,ACCOMMODATION_CODE,BOOKING_ID,BOOKING_STATUS,BOOKING_STATUS_DESCRIPTION,arrivaldate,departuredate,bookingdate,gbv,no_persons,no_child,no_adults
0,AT-5505-11,576245314,BOOKING,BOOKING,7/27/2019,8/3/2019,12/3/2018,2710.75,27,0,24
1,AT-5505-11,76234292,CANCELLED,CANCELLATION (NO PAYMENT),10/5/2019,10/12/2019,11/14/2018,0.00,19,10,9
2,AT-5505-11,1UHGKQGJ,BOOKING,BOOKING,10/5/2019,10/12/2019,8/21/2019,1839.04,23,12,11
3,AT-5505-11,776298327,CANCELLED,CANCELLATION WITHOUT INSURANCE,10/12/2019,10/19/2019,1/28/2019,567.00,24,17,7
4,AT-5541-33,76052288,BOOKING,BOOKING,3/2/2019,3/9/2019,3/7/2018,2288.59,19,6,11


In [4]:
bookings['arrivaldate'] = pd.to_datetime(bookings['arrivaldate'])
bookings['departuredate'] = pd.to_datetime(bookings['departuredate'])
bookings['bookingdate'] = pd.to_datetime(bookings['bookingdate'])

In [5]:
date_to_week = pd.read_csv(r"C:\Users\payoj.jain/Documents/projects/Europe Revenue estimation/data/weeks_start_date.csv")

In [6]:
date_to_week

,YEAR,WEEK,WK_START
0,2017,20,2017-05-13
1,2017,21,2017-05-20
2,2017,22,2017-05-27
3,2017,23,2017-06-03
4,2017,24,2017-06-10
...,...,...,...
185,2020,49,2020-11-28
186,2020,50,2020-12-05
187,2020,51,2020-12-12
188,2020,52,2020-12-19


In [7]:
date_to_week["WK_END"] = date_to_week["WK_START"]
date_to_week["WK_END"].values[:-1] = date_to_week["WK_START"].values[1:]
date_to_week["WK_END"].values[-1] = "NaN"

In [8]:
date_to_week

,YEAR,WEEK,WK_START,WK_END
0,2017,20,2017-05-13,2017-05-20
1,2017,21,2017-05-20,2017-05-27
2,2017,22,2017-05-27,2017-06-03
3,2017,23,2017-06-03,2017-06-10
4,2017,24,2017-06-10,2017-06-17
...,...,...,...,...
185,2020,49,2020-11-28,2020-12-05
186,2020,50,2020-12-05,2020-12-12
187,2020,51,2020-12-12,2020-12-19
188,2020,52,2020-12-19,2020-12-26


In [9]:
DE_bookings = bookings[(bookings["ACCOMMODATION_CODE"].str.contains("DE"))&(bookings["BOOKING_STATUS"]=="BOOKING")]

In [10]:
DE_bookings.head()

,ACCOMMODATION_CODE,BOOKING_ID,BOOKING_STATUS,BOOKING_STATUS_DESCRIPTION,arrivaldate,departuredate,bookingdate,gbv,no_persons,no_child,no_adults
5079,DE-00014-45,276245097,BOOKING,BOOKING,2019-05-29,2019-06-02,2018-12-03,1251.66,17,9,8
5080,DE-00014-45,976211446,BOOKING,BOOKING,2019-07-14,2019-07-21,2018-10-10,2457.10,19,5,14
5081,DE-00014-45,876136955,BOOKING,BOOKING,2018-08-10,2018-08-16,2018-07-03,1865.63,19,4,15
5082,DE-00016-24,876202668,BOOKING,BOOKING,2019-10-03,2019-10-06,2018-09-27,1438.96,24,4,20
5083,DE-00016-24,176212282,BOOKING,BOOKING,2019-09-13,2019-09-15,2018-10-11,1101.42,34,8,22


In [ ]:
day_wise_bookings = pd.DataFrame(columns=["ACCOMMODATION_CODE", "BOOKING_ID", "arrivaldate", "departuredate","bookingdate", "date", "revenue"])
count = 0
for booking in DE_bookings.values:
    if(count%10000==0):
        print(count)
    count+= 1
    days_of_booking = int(((booking[5]-booking[4]) / np.timedelta64(1, 'D')))
    rev_per_day = booking[7]/days_of_booking
    for i in range(days_of_booking):
        date = booking[4] + datetime.timedelta(i)
        booking_df = pd.DataFrame([(booking[0], booking[1], booking[4], booking[5], booking[6], date, rev_per_day)], columns=["ACCOMMODATION_CODE", "BOOKING_ID", "arrivaldate", "departuredate","bookingdate", "date", "revenue"])
        day_wise_bookings = day_wise_bookings.append(booking_df)

0
10000
20000
30000


In [ ]:
day_wise_bookings = days_wise_bookings.reset_index()

In [ ]:
day_wise_week = pd.DataFrame(columns=["year", "week", "week_start", "week_end","date"])
count = 0
for week in date_to_week.values:
    if(count%365==0):
        print(count)
    days_of_booking = int(((week[3]-week[2]) / np.timedelta64(1, 'D')))
#     rev_per_day = booking[7]/days_of_booking
    for i in range(days_of_booking):
        count+= 1
        date = week[2] + datetime.timedelta(i)
        week_df = pd.DataFrame([(week[0], week[1], week[2], week[3], date)], columns=["year", "week", "week_start", "week_end","date"])
        day_wise_week = day_wise_week.append(week_df)